<a href="https://colab.research.google.com/github/SheidaTalei/FinalProject/blob/main/SheidaTalei_Preprocessing_FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SUBJECT: Preprocessing 
## Author:  Sheida Talei

In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
nb_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0,nb_path)

Mounted at /content/drive


In [ ]:
!pip install --target=$nb_path hazm

  Using cached https://files.pythonhosted.org/packages/22/13/5a7074bc11d20dbbb46239349ac3f85f7edc148b4cf68e9b8c2f8263830c/hazm-0.7.0-py3-none-any.whl
Processing /root/.cache/pip/wheels/66/15/54/4510dce8bb958b1cdd2c47425cbd1e1eecc0480ac9bb1fb9ab/libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl
Processing /root/.cache/pip/wheels/d1/ab/40/3bceea46922767e42986aef7606a600538ca80de6062dc266c/nltk-3.3-cp37-none-any.whl
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
!pip install --target=$nb_path persianutils

  Using cached https://files.pythonhosted.org/packages/80/9c/5c382b9f45afc205e6c26cc94ed55b17949f3fa3a827f53571a6796257a4/persianutils-0.1.2-py3-none-any.whl


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
# from sklearn.feature_extraction.text import TfidfVectorizer
#Convert a collection of raw documents to a matrix of TF-IDF features.
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import regex
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from hazm import *
import re
import string
import persianutils as pu
import persianutils.PersianAlphabet as pALpha




In [ ]:
#------------------------------------------Empty Rows Removal----------------------------------------------------------------
# This function Will: 1- Remove all empty rows from csv file 2- Save data to the same csv
def removeEmptyRows(fileName):
    df = pd.read_csv(fileName, encoding='utf-8-sig')
    df = df.dropna(subset=['text'], how='all', axis=0) 
    df.to_csv(fileName, header=True, encoding='utf-8-sig',  index=False)
    
    return df

In [ ]:
#------------------------------------------------Normalization---------------------------------------------------------------
# This function will: 1- Normalize tweets (replace space with half-space)
def normalization (content):
    normalizer = Normalizer()
    tweets_normalize_list = []
    temp_set = []
    for item in range (len(content)):
        for word in content[item]:
            temp_word = normalizer.normalize(word)
            temp_set.append(temp_word.replace('\u200c', ' ', 1000000000))
                
                
        tweets_normalize_list.append(set(temp_set))
        temp_set = []
        
    return tweets_normalize_list

In [ ]:
#---------------------------------------------------- Latin and Punctuation Removal-------------------------------------------
# This function will: 1- Remove latin words 2- Remove punctuations
# Source: https://stackoverflow.com/questions/18429143/strip-punctuation-with-regex-python/50985687
def latinRemoval(content):
    only_persian_tweet_list =[]
    temp_set = []

    latin = re.compile(r'[a-zA-Z0-9@$!%*?&-_`.+#(»)«,:;،؛…..؟]+')  

    for item in range(len(content)):
        for word in content[item]:
            temp_word = word.replace('\u200c', ' ', 1000000000)
               
            if not(latin.match(temp_word)) :
                if not temp_word.isdigit():
                        #source: https://pypi.org/project/persianutils/
                    word = re.findall(r'[^۰-۹]+', temp_word.strip(string.punctuation))
                    
                    if len(word) >0:    
                        processed_text = pu.standardize4Word2vec(word[0].strip())

                        if (processed_text != ' '):
#                             print(processed_text)
                            word = re.findall('\d+', processed_text.strip())
                            if len(word)==0 : 
                                temp_set.append(processed_text.strip())
                                # print (processed_text.strip())
           
        only_persian_tweet_list.append(set(temp_set))
        temp_set = []
        
    return only_persian_tweet_list

In [ ]:
#--------------------------------------------------Loading StopWords ------------------------------------------
#Source of file: https://sites.google.com/site/kevinbouge/stopwords-lists
def getStopWord ():
    try:
        file = open('/content/drive/MyDrive/Final/stopwords_fa.txt', 'r', encoding='utf-8-sig')
        file_readed = file.read()
    
    finally:
        file.close()
        
    stopWord_Set = set(file_readed.split())
    return stopWord_Set


In [ ]:
#------------------------------------Tokenization and Stop Word Removal------------------------------------------------------
# This function will: 1- Tokenize words  2- Remove stop words and return a list of sets
def getTweetNoStopWord(content):
    tweet_no_StopWord_list =[]
    temp_set = []
    stop_word_list = getStopWord()
    for item in range(len(content)):
        for word in content[item]:
            if not word in stop_word_list:
                temp_set.append(word.replace('\u200c', ' ', 1000000000))
                
        tweet_no_StopWord_list.append(set (temp_set))
        temp_set = []
        
    return tweet_no_StopWord_list

In [ ]:
#----------------------------------------------Lemmatization ------------------------------------------------------------------
def Lemmatization (content):
    lemmatizer = Lemmatizer()
    tweets_lemmatize_list = []
    temp_set = []
    for item in range(len(content)):
        for word in (content[item]):
            temp_set.append(lemmatizer.lemmatize(word.replace('\u200c', ' ', 1000000000)))
        tweets_lemmatize_list.append(set(temp_set))
        temp_set = []
    
    return tweets_lemmatize_list

In [ ]:
#---------------------------------------------------Tokenization-------------------------------------------------------------
# Tokenizing X and at the end append Y as a label
def tokenization (X):
    tokenized_train_data = []
    for item in range(len(X)):
        var = word_tokenize(X[item])
#         var.append(Y[item])
        tokenized_train_data.append(var)
        
    return tokenized_train_data    

In [ ]:
#Loading data

# dfPrime_1 = pd.read_csv ('/content/drive/MyDrive/Final/Prime#1.csv', encoding='utf-8-sig')
# dfPrime_2 = pd.read_csv ('/content/drive/MyDrive/Final/Prime#2.csv', encoding='utf-8-sig')
# dfPrime_3 = pd.read_csv ('/content/drive/MyDrive/Final/Prime#3.csv', encoding='utf-8-sig')


# data_frame =  pd.read_csv ('/content/drive/MyDrive/Final/Prepared_Data_1_Labeled.csv', encoding='utf-8-sig')
# df2 =  pd.read_csv ('/content/drive/MyDrive/Final/Prepared_test_Data_test#1.csv', encoding='utf-8-sig')

# df2 = pd.read_csv ('/content/drive/MyDrive/Final/Main#33.csv', encoding='utf-8-sig')
# df3 = pd.read_csv ('/content/drive/MyDrive/Final/Main#34.csv', encoding='utf-8-sig')
# df4 = pd.read_csv ('/content/drive/MyDrive/Final/Main#35.csv', encoding='utf-8-sig')
# df5 = pd.read_csv ('/content/drive/MyDrive/Final/Main#36.csv', encoding='utf-8-sig')


df1= removeEmptyRows ('/content/drive/MyDrive/Final/train_temp.csv')
df2 =  removeEmptyRows ('/content/drive/MyDrive/Final/test_temp.csv')
# data_frame = removeEmptyRows('/content/drive/MyDrive/Final/test_temp.csv')

data_frame= pd.concat([df1, df2 ], ignore_index=True, names=['tweeter_handle', 'text', 'Label', 'statues_count' ])


In [ ]:
#----------------------------------------------------------Step1----------------------------------------------------------
#----------------------------------Normalization_StopWord_Lemmatization_Punctuaation--------------------------------------
#Reading just coulmns of description and Label
# Remove repetetive tweets
def cleanData (df):
    df.drop_duplicates(subset=['text', 'tweeter_handle'], inplace=True)
    # train_data, test_data = train_test_split(df, test_size=0.2)
    # print('train data size: {}    test data size: {}'.format(len(train_data), len(test_data)))
    header = ['tweeter_handle', 'text', 'Label', 'statues_count']
    df.to_csv('/content/drive/MyDrive/Final/X_and_Y_test#1.csv', index=False, encoding='utf-8-sig' , columns = header )
    


    #Step 1: Removing empty rows from data set which contains only description and Label
    # This functon will remove empty rows and save them to same file (input file)
    removeEmptyRows('/content/drive/MyDrive/Final/X_and_Y_test#1.csv')
    
    #test
    # test_data.to_csv('/content/drive/MyDrive/Final/X_and_Y_test.csv', index=False, encoding='utf-8-sig')
    # #Step 1: Removing empty rows from data set which contains only description and Label
    # # This functon will remove empty rows and save them to same file (input file)
    # removeEmptyRows('/content/drive/MyDrive/Final/X_and_Y_test.csv')
    
    return df

#---------------------------------------------test-------------------------------------------------------------------------



In [ ]:
cleanData (data_frame)


In [ ]:
#------------------------------------------------------Step 2------------------------------------------------------------------
#step 2: Tokenizing data
X_and_Y = pd.read_csv('/content/drive/MyDrive/Final/X_and_Y_test#1.csv',encoding="utf-8")



X = X_and_Y.text
Y= X_and_Y.Label


#Tokenization
tokenize_list = tokenization (X)
#------------------------------------------------------test---------------------------------------------------------------------
#step 2: Tokenizing data
# X_and_Y_test = pd.read_csv('/content/drive/MyDrive/Final/X_and_Y_test.csv',encoding="utf-8")



# X_test = X_and_Y_test.text
# Y_test= X_and_Y_test.Label

# # #Tokenization
# train_data_tokenize_list_test = tokenization (X_test)

In [ ]:
print(tokenize_list[3755])

In [ ]:
#------------------------------------------------------Step 3------------------------------------------------------------------
#step 3: removing HASHTAGS, MENTIONS, NUMBERS and any LATIN WORD
# This function get tokenized list of data and remove mentioned elements

only_persian_tweet_list = latinRemoval(tokenize_list)
#------------------------------------------------------test--------------------------------------------------------------------
#step 3: removing HASHTAGS, MENTIONS, NUMBERS and any LATIN WORD
# This function get tokenized list of data and remove mentioned elements

# only_persian_tweet_list_test = latinRemoval(train_data_tokenize_list_test)

In [ ]:
print(tokenize_list[3755])
print ('  ')
print(only_persian_tweet_list[3755])

In [ ]:
#------------------------------------------------------Step 4------------------------------------------------------------------
#step 4: Normalization
# normalize_tweet_list = normalization (only_persian_tweet_list)

#-------------------------------------------------------test-------------------------------------------------------------------
#step 4: Normalization
# normalize_tweet_list_test = normalization (only_persian_tweet_list_test)


In [ ]:
# print(normalize_tweet_list[3755])

In [ ]:
#------------------------------------------------------Step 5------------------------------------------------------------------
#step 5: Remove stop words
noStopWord_tweet_List = getTweetNoStopWord(only_persian_tweet_list)

#-------------------------------------------------------test-------------------------------------------------------------------
#step 5: Remove stop words
# noStopWord_tweet_List_test = getTweetNoStopWord(normalize_tweet_list_test)


In [ ]:
print(noStopWord_tweet_List[3755])

In [ ]:
#------------------------------------------------------Step 6------------------------------------------------------------------
#step 6: Lemmatixation
lemmatize_tweet_list = Lemmatization (noStopWord_tweet_List)
#-------------------------------------------------------test-------------------------------------------------------------------
#step 6: Lemmatixation
# lemmatize_tweet_list_test = Lemmatization (noStopWord_tweet_List_test)


In [7]:
#Source : https://stackabuse.com/python-for-nlp-movie-sentiment-analysis-using-deep-learning-in-keras/
#shows the proportion of the whole data both train and test
# import seaborn as sns

# sns.countplot(x='Label', data=cleanData (data_frame))

In [ ]:
def convertSequenceWordsToText(input_list,  fileName, lastFileName):
    X_and_Y = pd.read_csv(fileName , encoding="utf-8")
    text = ''
    
    counter = 0
    updator = 0
    for item in range(len(input_list)):
      if counter <= 80000:
        for i in range(len(input_list [item + updator])):
          text += list(input_list[item + updator])[i] + ' '
      else:
        break
      
      print(counter)    
      X_and_Y.loc [counter, 'text'] = text.strip()
      X_and_Y.to_csv(lastFileName ,index=False, encoding='utf-8-sig')
      counter = counter + 1
      text =''
    print("This is counter: " ,counter-1)

In [ ]:
# load additional module
import pickle

# define a list of places

file_name = "/content/drive/MyDrive/Final/lemmatize_tweet_list_test#1.pkl"
open_file = open(file_name, "wb")
pickle.dump(lemmatize_tweet_list, open_file)
open_file.close()

file_name = "/content/drive/MyDrive/Final/lemmatize_tweet_list_test#1.pkl"
open_file = open(file_name, "rb")
lemmatize_tweet_list = pickle.load(open_file)
open_file.close()



In [ ]:
print(lemmatize_tweet_list[0])

{'خدا', 'حالا', 'ایران', 'معطل', 'کنه', 'جمهوریخواه', 'مانده', 'حرومزاده', 'لعنت', 'انداخت#انداز', 'انقلاب', 'کردن', 'دموکرات', 'الاغ', 'فیل', 'تحریم'}


In [ ]:
convertSequenceWordsToText(lemmatize_tweet_list, '/content/drive/MyDrive/Final/X_and_Y_test#1.csv', '/content/drive/MyDrive/Final/Prepared_Data_test#1.csv' )
# convertSequenceWordsToText(lemmatize_tweet_list,'/content/drive/MyDrive/Final/Prepared_Data_test#1.csv', '/content/drive/MyDrive/Final/Prepared_Data_test#1.csv')

In [ ]:

# convertSequenceWordsToText(lemmatize_tweet_list_test, '/content/drive/MyDrive/Final/X_and_Y_test.csv', '/content/drive/MyDrive/Final/Prepared_test_Data.csv' )
# convertSequenceWordsToText(lemmatize_tweet_list_test,'/content/drive/MyDrive/Final/Prepared_test_Data.csv', '/content/drive/MyDrive/Final/Prepared_test_Data.csv' )


In [ ]:
#divide data to train and test
dataFrame =  pd.read_csv ('/content/drive/MyDrive/Final/Prepared_Data_test#1.csv', encoding='utf-8-sig')
train_data, test_data = train_test_split(dataFrame, test_size=0.2)
print('train data size: {}    test data size: {}'.format(len(train_data), len(test_data)))
train_data.to_csv('/content/drive/MyDrive/Final/Prepared_train_Data_test#1.csv' ,index=False, encoding='utf-8-sig')
test_data.to_csv('/content/drive/MyDrive/Final/Prepared_test_Data_test#1.csv' ,index=False, encoding='utf-8-sig')

In [ ]:
count_0 = 0
count_1 = 0
Y = test_data.Label
for i in Y:
    if (i==0):
        count_0 = count_0+1
    else:
        count_1 = count_1 + 1

In [ ]:
print("count_0: ",count_0)
print("count_1: ",count_1)

count_0:  29044
count_1:  5223
